# Computing expected return of various portfolios using Fama French 3 factor model

In [23]:
import pandas as pd
import pandas_datareader.data as reader
import datetime as dt
import statsmodels.api as sm
import matplotlib.pyplot as plt
import getFamaFrenchFactors as gff
import seaborn as sns

## Portfolio 1: Large Cap Growth Stock(Tesla)

In [24]:
end = dt.datetime.now()
start = dt.date(end.year-6,end.month,end.day) #getting data from past 6 years
ticker = ['TSLA']

In [25]:
stock_prices = reader.get_data_yahoo(ticker,start,end)['Adj Close'] #getting data from yahoo finance
stock_prices

Symbols,TSLA
Date,
2015-12-30,47.618000
2015-12-31,48.001999
2016-01-04,44.681999
2016-01-05,44.686001
2016-01-06,43.807999
...,...
2021-12-23,1067.000000
2021-12-27,1093.939941
2021-12-28,1088.469971


In [26]:
stock_prices = stock_prices.resample('1M').last()
stock_prices

Symbols,TSLA
Date,
2015-12-31,48.001999
2016-01-31,38.240002
2016-02-29,38.386002
2016-03-31,45.953999
2016-04-30,48.152000
...,...
2021-08-31,735.719971
2021-09-30,775.479980
2021-10-31,1114.000000


In [27]:
stock_returns = stock_prices.pct_change().dropna()
stock_returns.head()

Symbols,TSLA
Date,
2016-01-31,-0.203366
2016-02-29,0.003818
2016-03-31,0.197155
2016-04-30,0.047830
2016-05-31,-0.072811


In [28]:
ff3_monthly = pd.DataFrame(gff.famaFrench3Factor(frequency = 'm'))
ff3_monthly.rename(columns = {'date_ff_factors':'Date'},inplace = True)
ff3_monthly.set_index('Date', inplace = True)
ff3_monthly

,Mkt-RF,SMB,HML,RF
Date,,,,
1926-07-31,0.0296,-0.0238,-0.0273,0.0022
1926-08-31,0.0264,-0.0147,0.0414,0.0025
1926-09-30,0.0036,-0.0139,0.0012,0.0023
1926-10-31,-0.0324,-0.0013,0.0065,0.0032
1926-11-30,0.0253,-0.0016,-0.0038,0.0031
...,...,...,...,...
2021-06-30,0.0275,0.0180,-0.0776,0.0000
2021-07-31,0.0127,-0.0396,-0.0175,0.0000
2021-08-31,0.0290,-0.0048,-0.0013,0.0000


In [46]:
data = ff3_monthly.merge(stock_returns , on = 'Date')
data

,Mkt-RF,SMB,HML,RF,TITN
Date,,,,,
2016-01-31,-0.0577,-0.0339,0.0207,0.0001,-0.223239
2016-02-29,-0.0008,0.0081,-0.0057,0.0002,0.121319
2016-03-31,0.0696,0.0075,0.0110,0.0002,0.214286
2016-04-30,0.0092,0.0067,0.0321,0.0001,0.124567
2016-05-31,0.0178,-0.0019,-0.0165,0.0001,-0.179231
...,...,...,...,...,...
2021-06-30,0.0275,0.0180,-0.0776,0.0000,0.008475
2021-07-31,0.0127,-0.0396,-0.0175,0.0000,-0.077893
2021-08-31,0.0290,-0.0048,-0.0013,0.0000,0.006660


In [30]:
TSLA_excess_return = data['TSLA'] - data['RF']
data['TSLA-RF'] = TSLA_excess_return
data.head()

,Mkt-RF,SMB,HML,RF,TSLA,TSLA-RF
Date,,,,,,
2016-01-31,-0.0577,-0.0339,0.0207,0.0001,-0.203366,-0.203466
2016-02-29,-0.0008,0.0081,-0.0057,0.0002,0.003818,0.003618
2016-03-31,0.0696,0.0075,0.0110,0.0002,0.197155,0.196955
2016-04-30,0.0092,0.0067,0.0321,0.0001,0.047830,0.047730
2016-05-31,0.0178,-0.0019,-0.0165,0.0001,-0.072811,-0.072911


In [31]:
X = data[['Mkt-RF','SMB','HML']]
y = data['TSLA-RF']

X1 = sm.add_constant(X)
model = sm.OLS(y,X1)

results = model.fit()
results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                TSLA-RF   R-squared:                       0.285
Model:                            OLS   Adj. R-squared:                  0.252
Method:                 Least Squares   F-statistic:                     8.766
Date:                Fri, 31 Dec 2021   Prob (F-statistic):           5.66e-05
Time:                        19:52:30   Log-Likelihood:                 29.872
No. Observations:                  70   AIC:                            -51.74
Df Residuals:                      66   BIC:                            -42.75
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.0249      0.021      1.209      0.231      -0.016       0.066
Mkt-RF         2.2966      0.467      4.916      0.000       1.364       3.229
SMB           -0.4033      0.790     -0.510      0.612      -1.981       1.175
HML           -0.9530      0.551     -1.729      0.089      -2.054       0.148
==============================================================================
Omnibus:                       14.786   Durbin-Watson:                   1.740
Prob(Omnibus):                  0.001   Jarque-Bera (JB):               16.916
Skew:                           1.000   Prob(JB):                     0.000212
Kurtosis:                       4.342   Cond. No.                         42.0
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [32]:
intercept,beta_m,beta_s,beta_v = results.params
print(beta_m,beta_s,beta_v)

2.296647208884073 -0.4032518164108486 -0.952976547924055


## Estimating expected return

In [33]:
risk_free = data['RF'].mean()
risk_free

0.000765714285714286

In [34]:
market_premium = ff3_monthly['Mkt-RF'].mean()
size_premium = ff3_monthly['SMB'].mean()
value_premium = ff3_monthly['HML'].mean()
print(market_premium,size_premium,value_premium)

0.006931905594405599 0.0020048076923076938 0.003346678321678321


### E(R_tesla) = R_f + beta_m * market_premium + beta_s * size_premium + beta_v * value_premium 


In [35]:
exp_monthly_return = risk_free + beta_m*market_premium + beta_s*size_premium + beta_v*value_premium 
exp_yearly_return = 12*exp_monthly_return
exp_yearly_return

0.15225729148645195

## Portfolio 2: Small Cap Value Stocks(Titan Machinery Inc.)

In [36]:
end = dt.datetime.now()
start = dt.date(end.year-6,end.month,end.day) #getting data from past 6 years
ticker = ['TITN']

In [37]:
stock_prices = reader.get_data_yahoo(ticker,start,end)['Adj Close'] #getting data from yahoo finance
stock_prices

Symbols,TITN
Date,
2015-12-30,10.990000
2015-12-31,10.930000
2016-01-04,10.560000
2016-01-05,10.330000
2016-01-06,10.680000
...,...
2021-12-23,32.180000
2021-12-27,33.279999
2021-12-28,33.580002


In [38]:
stock_prices = stock_prices.resample('1M').last()
stock_prices.head()

Symbols,TITN
Date,
2015-12-31,10.93
2016-01-31,8.49
2016-02-29,9.52
2016-03-31,11.56
2016-04-30,13.00


In [39]:
stock_returns = stock_prices.pct_change().dropna()
stock_returns.head()

Symbols,TITN
Date,
2016-01-31,-0.223239
2016-02-29,0.121319
2016-03-31,0.214286
2016-04-30,0.124567
2016-05-31,-0.179231


In [47]:
data = ff3_monthly.merge(stock_returns , on = 'Date')
data

,Mkt-RF,SMB,HML,RF,TITN
Date,,,,,
2016-01-31,-0.0577,-0.0339,0.0207,0.0001,-0.223239
2016-02-29,-0.0008,0.0081,-0.0057,0.0002,0.121319
2016-03-31,0.0696,0.0075,0.0110,0.0002,0.214286
2016-04-30,0.0092,0.0067,0.0321,0.0001,0.124567
2016-05-31,0.0178,-0.0019,-0.0165,0.0001,-0.179231
...,...,...,...,...,...
2021-06-30,0.0275,0.0180,-0.0776,0.0000,0.008475
2021-07-31,0.0127,-0.0396,-0.0175,0.0000,-0.077893
2021-08-31,0.0290,-0.0048,-0.0013,0.0000,0.006660


In [41]:
TITN_excess_return = data['TITN'] - data['RF']
data['TITN-RF'] = TITN_excess_return
data.head()

,Mkt-RF,SMB,HML,RF,TITN,TITN-RF
Date,,,,,,
2016-01-31,-0.0577,-0.0339,0.0207,0.0001,-0.223239,-0.223339
2016-02-29,-0.0008,0.0081,-0.0057,0.0002,0.121319,0.121119
2016-03-31,0.0696,0.0075,0.0110,0.0002,0.214286,0.214086
2016-04-30,0.0092,0.0067,0.0321,0.0001,0.124567,0.124467
2016-05-31,0.0178,-0.0019,-0.0165,0.0001,-0.179231,-0.179331


In [42]:
X = data[['Mkt-RF','SMB','HML']]
y = data['TITN-RF']

X1 = sm.add_constant(X)
model = sm.OLS(y,X1)

results = model.fit()
results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                TITN-RF   R-squared:                       0.436
Model:                            OLS   Adj. R-squared:                  0.411
Method:                 Least Squares   F-statistic:                     17.03
Date:                Fri, 31 Dec 2021   Prob (F-statistic):           2.69e-08
Time:                        19:52:31   Log-Likelihood:                 52.060
No. Observations:                  70   AIC:                            -96.12
Df Residuals:                      66   BIC:                            -87.13
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.0065      0.015      0.433      0.666      -0.023       0.036
Mkt-RF         1.3796      0.340      4.054      0.000       0.700       2.059
SMB            1.7724      0.576      3.079      0.003       0.623       2.922
HML            0.8792      0.402      2.189      0.032       0.077       1.681
==============================================================================
Omnibus:                        0.535   Durbin-Watson:                   2.167
Prob(Omnibus):                  0.765   Jarque-Bera (JB):                0.543
Skew:                           0.198   Prob(JB):                        0.762
Kurtosis:                       2.830   Cond. No.                         42.0
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [43]:
intercept,beta_m,beta_s,beta_v = results.params
print(beta_m,beta_s,beta_v)

1.3796471156679422 1.7723805771511132 0.8791827485882489


### E(R_titn) = R_f + beta_m * market_premium + beta_s * size_premium + beta_v * value_premium 

In [45]:
exp_monthly_return = risk_free + beta_m*market_premium + beta_s*size_premium + beta_v*value_premium 
exp_yearly_return = 12*exp_monthly_return
exp_yearly_return

0.2018990628645792

#### Conclusion: Portfolio 1 which has a large cap growth stock gives an expected return of 15.2% as of 31-10-2021
####                       Portfolio 2 which has a small cap value stock gives an expected  return of 20.2% as of 31-10-2021